<a href="https://colab.research.google.com/github/Dhanraj7573/Python-Projects/blob/main/clean_up_messy_expense_claims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# --- Sample "messy" expense data ---
data = [
    {"expense_id":"E001","employee":"Dylan","date":"2026-01-03","merchant":"uber ","category_raw":" travel ","amount_gbp":"£12.50","status":"Reimbursed","reimbursed_date":"2026-01-07"},
    {"expense_id":"E002","employee":"Dylan","date":"03/01/2026","merchant":"Pret A Manger","category_raw":"Meals&Ent","amount_gbp":"8.90","status":"reimbursed","reimbursed_date":"2026-01-08"},
    {"expense_id":"E003","employee":"Asha","date":"2026/01/05","merchant":"Amazon","category_raw":"Office","amount_gbp":"£34.99","status":"Pending","reimbursed_date":None},
    {"expense_id":"E004","employee":"Sam","date":"2026-02-30","merchant":"trainline","category_raw":"TRVL","amount_gbp":"£45.00","status":"Paid","reimbursed_date":None},  # invalid date + status
    {"expense_id":"E005","employee":"Asha","date":"2026-01-09","merchant":"UBER","category_raw":"Travel","amount_gbp":"-£6.00","status":"Reimbursed","reimbursed_date":"2026-01-10"}, # negative (refund?) but category travel
    {"expense_id":"E006","employee":"Sam","date":"2027-01-01","merchant":"Starbucks","category_raw":"Meals","amount_gbp":"£5.25","status":"Pending","reimbursed_date":None}, # future date
    {"expense_id":"E007","employee":"Dylan","date":"2026-01-10","merchant":"WeWork","category_raw":"Office supplies","amount_gbp":"£120","status":"Reimbursed","reimbursed_date":None}, # missing reimbursed_date
    {"expense_id":"E008","employee":"Asha","date":"2026-01-11","merchant":"tesco","category_raw":"misc","amount_gbp":"£14.10","status":"Cancelled","reimbursed_date":None},
    {"expense_id":"E009","employee":"Sam","date":"2026-01-11","merchant":None,"category_raw":"Travel","amount_gbp":"£22.00","status":"Pending","reimbursed_date":None}, # missing merchant
    {"expense_id":"E010","employee":"Dylan","date":"2026-01-12","merchant":"Amazon","category_raw":"OFFICE","amount_gbp":"£19.99","status":"Pending","reimbursed_date":None},
]

df = pd.DataFrame(data)

#  A: clean text columns (categoricals start here)
for col in ["merchant", "category_raw", "status"]:
    df[col] = df[col].astype("string").str.strip()

df["merchant_clean"] = df["merchant"].str.lower()
df["status_clean"] = df["status"].str.lower()

#  B: parse dates + amount properly
df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=True)
df["reimbursed_date"] = pd.to_datetime(df["reimbursed_date"], errors="coerce")

df["amount_gbp"] = (
    df["amount_gbp"]
      .astype("string")
      .str.replace("£", "", regex=False)
      .str.replace(",", "", regex=False)
)
df["amount_gbp"] = pd.to_numeric(df["amount_gbp"], errors="coerce")

# C: map messy categories into standard categories
category_map = {
    "travel":"Travel",
    "trvl":"Travel",
    "meals":"Meals",
    "meals&ent":"Meals",
    "office":"Office",
    "office supplies":"Office",
    "off ice":"Office",
    "misc":"Other",
}
df["category_key"] = df["category_raw"].str.lower().str.replace(r"[^a-z& ]", "", regex=True).str.strip()
df["category_clean"] = df["category_key"].map(category_map).fillna("Other")

#   D: data quality rules (flags)
allowed_status = {"pending", "reimbursed", "cancelled"}

df["flag_missing_required"] = df[["expense_id","employee","merchant_clean","date","amount_gbp"]].isna().any(axis=1)
df["flag_bad_status"] = ~df["status_clean"].isin(allowed_status)
df["flag_future_date"] = df["date"] > pd.Timestamp("today").normalize()
df["flag_bad_amount"] = df["amount_gbp"].isna() | (df["amount_gbp"] == 0)

# Business rule: if reimbursed, reimbursed_date must exist
df["flag_reimbursed_missing_date"] = (df["status_clean"] == "reimbursed") & (df["reimbursed_date"].isna())

# Business rule: negative amounts should usually be Refunds (simple rule for demo)
df["flag_negative_not_refund"] = (df["amount_gbp"] < 0) & (df["category_clean"] != "Other")

# Single "needs review" flag
flag_cols = [c for c in df.columns if c.startswith("flag_")]
df["needs_review"] = df[flag_cols].any(axis=1)

# E: clean report table (only valid rows)
clean = df[~df["needs_review"]].copy()

summary = (
    clean.groupby("category_clean", as_index=False)
         .agg(total_spend=("amount_gbp","sum"),
              txns=("expense_id","count"))
         .sort_values("total_spend", ascending=False)
)

print("=== Rows needing review ===")
print(df.loc[df["needs_review"], ["expense_id","employee","category_raw","status","date","amount_gbp","needs_review"]])

print("\n=== Clean summary (ready for reporting) ===")
print(summary)


=== Rows needing review ===
  expense_id employee     category_raw      status       date  amount_gbp  \
0       E001    Dylan           travel  Reimbursed 2026-03-01        12.5   
1       E002    Dylan        Meals&Ent  reimbursed        NaT         8.9   
2       E003     Asha           Office     Pending        NaT       34.99   
3       E004      Sam             TRVL        Paid        NaT        45.0   
4       E005     Asha           Travel  Reimbursed 2026-09-01        -6.0   
5       E006      Sam            Meals     Pending 2027-01-01        5.25   
6       E007    Dylan  Office supplies  Reimbursed 2026-10-01       120.0   
7       E008     Asha             misc   Cancelled 2026-11-01        14.1   
8       E009      Sam           Travel     Pending 2026-11-01        22.0   
9       E010    Dylan           OFFICE     Pending 2026-12-01       19.99   

   needs_review  
0          True  
1          True  
2          True  
3          True  
4          True  
5          True 